A和B实际访存模式提取

In [2]:
!python /home/shiroha/Code/Backend/SpMM-simulator/spmmsim/model/hw_model/compute_model/sparse_unit/interface.py 
!cp /home/shiroha/Code/Backend/SpMM-simulator/spmmsim/model/hw_model/compute_model/sparse_unit/output/*.txt ./data/

[TEST1][one-side-sparse, indirection, col_dim_skip, horizontal_tile, 16x16] 



#### 数据预处理
提取五个信息矩阵
- 提取A访问矩阵，
- 提取A的访问区间，
- 提取B访问矩阵，
- 提取A的带索引访问矩阵，
- 提取B的带索引访问矩阵，

In [1]:
import pandas as pd
import numpy as np
import re

acceess_file_A        = './data/TEST1_A_access.txt'  
acceess_file_B        = './data/TEST1_B_access.txt'  
acceess_interval_file = './data/TEST1_A_access_interval.txt'  

######################## 提取A访问矩阵
with open(acceess_file_A, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, y, z) 中的z提取
def extract_func_1(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        zs = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', last_match)
        return [int(t[2]) for t in zs]
    return []

# 提取每行最后一个方括号内的内容
access_matrix_a = []
for line in lines:
    extracted_data = extract_func_1(line)
    access_matrix_a.append(extracted_data)

# 将提取的数据转换为 NumPy 数组，每行 16 个元素，不足的补 -1
max_length = 64
access_matrix_np_a = np.array([data + [-1] * (max_length - len(data)) if len(data) < max_length else data[:max_length] for data in access_matrix_a])
# access_matrix
# print(access_matrix_np.tolist())


######################## 提取ss_interval #######################
with open(acceess_interval_file, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (start, end) 中的start或end提取
def extract_interval(text, idx):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        intervals = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        # print(intervals)
        return [int(t[idx]) for t in intervals]
    return []


start_matrix = []
end_matrix = []
for line in lines:
    start = extract_interval(line, 0)
    end = extract_interval(line, 1)
    start_matrix.append(start)
    end_matrix.append(end)

######################## 提取B访问矩阵
with open(acceess_file_B, 'r') as file:
    lines = file.readlines()

# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, z) 中的z提取
def extract_func_2(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        zs = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        return [int(t[1]) for t in zs]
    return []

# 提取每行最后一个方括号内的内容
access_matrix_b = []
for line in lines:
    extracted_data = extract_func_2(line)
    access_matrix_b.append(extracted_data)

# 将提取的数据转换为 NumPy 数组，每行 64 个元素，不足的补 -1
max_length = 64 # 脑子抽了，下面-1记得扔
access_matrix_np_b = np.array([data + [-1] * (max_length - len(data)) if len(data) < max_length else data[:max_length] for data in access_matrix_b])


######################## 提取A的索引访问矩阵
with open(acceess_file_A, 'r') as file:
    lines = file.readlines()
    
# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, y, z) 提取
def extract_func_3(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        tuples = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', last_match)
        return [tuple(map(int, t)) for t in tuples]
    return []

access_idx_matrix_a = []
for idx, line in enumerate(lines):
    extracted_data = extract_func_3(line)
    access_idx_matrix_a.append((idx, extracted_data))


######################## 提取B的索引访问矩阵
with open(acceess_file_B, 'r') as file:
    lines = file.readlines()
    
# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, z) 提取
def extract_func_4(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        tuples = re.findall(r'\((\d+),\s*(\d+)\)', last_match)
        return [tuple(map(int, t)) for t in tuples]
    return []

access_idx_matrix_b = []
for idx, line in enumerate(lines):
    extracted_data = extract_func_4(line)
    access_idx_matrix_b.append((idx, extracted_data))



预取访存数据提取

In [2]:
!pwd
!rm ./output/*.txt

/home/shiroha/Code/Backend/SpMM-simulator/spmmsim/model/hw_model/playground


In [3]:
import sys
sys.path.append("../")

from system_model.prefetcher.SP.stride_prefetch import StridePrefetcherGroup
from system_model.prefetcher.IMP.imp import IMPGroup
from system_model.prefetcher.DVR.DVR import DVR
from system_model.prefetcher.NVR.NVR import NVR

# 这个函数应该究极耗时
# def A_idx_B(a_num, A_matrix, B_matrix, num_has_get):
#     # num_has_get=(i, num_has_get)
#     # 是指这一轮之前已经在i这里取了几个数了，这次应该在第num_has_get+1个地方取
#     for a_idx, data in A_matrix:
#         for row, col, a_addr in data:
#             if a_num == a_addr:
#                 b_idx, data = B_matrix[a_idx]
#                 b_addr_matched_set = [b_addr for i, b_addr in data if i == row]
#                 # for i, b_addr in data:
#                 #     if i == row:
#                 num_to_get = num_has_get[i][1]+1
#                 return b_addr_matched_set[num_to_get], 
#     return -1

from collections import defaultdict

def A_idx_B(a_num, A_matrix, B_matrix, record_buffer=None):
    # 使用 defaultdict 来存储已经取过的值
    if record_buffer is None:
        record_buffer = defaultdict(list)
    
    # 遍历 A_matrix 查找 a_num 对应的地址
    for a_idx, data in A_matrix:
        for row, col, a_addr in data:
            if a_num == a_addr:
                # # 获取对应 B_matrix 的数据
                # b_idx, b_data = B_matrix[a_idx]
                # # 找到匹配的行号的所有 b_addr
                # b_addr_matched_set = [b_addr for i, b_addr in b_data if i == col]
                
                # # 检查在缓冲区中已经记录的索引
                # already_taken = len(record_buffer[(a_idx, col)])
                # if already_taken < len(b_addr_matched_set):
                #     # 记录取过的值
                #     record_buffer[(a_idx, col)].append(b_addr_matched_set[already_taken])
                #     return b_addr_matched_set[already_taken]
                # else:
                    # return -1
                return col
    return -1

def clear_buffer(record_buffer):
    # 清空记录缓冲区
    record_buffer.clear()
    

if __name__ == "__main__":
    sp = StridePrefetcherGroup(num_prefetchers=64, default_stride=4)
    imp = IMPGroup(num_units=16)
    # imp = DVR(lbd=False, vector_size=16, ewma_alpha=0.3, initial_bound=2)
    dvr = DVR(lbd=True, vector_size=16, ewma_alpha=0.3, initial_bound=2)
    nvr = NVR()
    

    mvin_instructions = access_matrix_np_a
    ss_start = start_matrix
    ss_end = end_matrix
    

    # bound = [[end - start for start, end in zip(start_list, end_list)] for start_list, end_list in zip(ss_start, ss_end)]
    bound = [[
            (end - start + 1) if (start != 0 and end != 0) else 0
            for start, end in zip(start_list, end_list)
        ]
        for start_list, end_list in zip(ss_start, ss_end)
    ]
    # print(bound)
    
    ## 索引矩阵
    idx_matrix_a = access_idx_matrix_a
    idx_matrix_b = access_idx_matrix_b

    
    # tiled_num = 3
    for i in range(len(access_matrix_np_a)-1):
    # for i in range(3):
        # print(mvin_instructions[i])
        sp_predict_addr  =  sp.prefetch(mvin_instructions[i]) 
        imp_predict_addr = imp.prefetch(bound[i], ss_start[i+1]) 
        dvr_predict_addr = dvr.prefetch(bound[i], ss_start[i+1]) 
        nvr_predict_addr = nvr.prefetch(ss_start[i+1], ss_end[i+1]) 
        actrual_addr     = mvin_instructions[i+1]
        

        np.set_printoptions(threshold=np.inf)
        prefetch_buffer = defaultdict(list)
        ### 步长预取器结果 ######################################
        sp_result_a = []
        for _, _, predicted_address, _ in sp_predict_addr:
            sp_result_a.append(int(predicted_address))
        with open(f"./output/sp_A_predict_addr.txt", "a") as file:
            file.write(f"{sp_result_a}\n")  
        print(f"sp预测A地址{sp_result_a}" )
        ########################################################
        
        ### IMP预取器结果 ######################################
        imp_result_a = []
        for _, predicted_address in imp_predict_addr:
            imp_result_a.append(int(predicted_address))
        with open(f"./output/imp_A_predict_addr.txt", "a") as file:
            file.write(f"{imp_result_a}\n")  
        print(f"imp预测A地址{imp_result_a}") # imp预测A并不是为了预取A, 它只是根据这个结果A[i]取B[A[i]](吗?)

        imp_result_b = []
        for a_addr in imp_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            imp_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/imp_B_predict_addr.txt", "a") as file:
            file.write(f"{imp_result_b}\n")  
        print(f"imp预测B地址{imp_result_b}")
        ########################################################

        ### DVR预取结果 ######################################
        dvr_result_a = []
        for _, predicted_address in dvr_predict_addr:
            dvr_result_a.append(int(predicted_address))
        with open(f"./output/dvr_A_predict_addr.txt", "a") as file:
            file.write(f"{dvr_result_a}\n")  
        print(f"dvr预测A地址{dvr_result_a}")

        dvr_result_b = []
        for a_addr in dvr_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            dvr_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/dvr_B_predict_addr.txt", "a") as file:
            file.write(f"{dvr_result_b}\n")  
        print(f"dvr预测B地址{dvr_result_b}")
        ########################################################

        ### NVR预取结果 ######################################
        nvr_result_a = []
        for _, predicted_address in nvr_predict_addr:
            nvr_result_a.append(int(predicted_address))
        with open(f"./output/nvr_A_predict_addr.txt", "a") as file:
            file.write(f"{nvr_result_a}\n")  
        print(f"nvr预测A地址{nvr_result_a}")
        
        nvr_result_b = []
        for a_addr in nvr_result_a:
            b_addr = A_idx_B(a_addr, idx_matrix_a, idx_matrix_b, prefetch_buffer)
            nvr_result_b.append(b_addr)
        clear_buffer(prefetch_buffer)
        with open(f"./output/nvr_B_predict_addr.txt", "a") as file:
            file.write(f"{nvr_result_b}\n")  
        print(f"nvr预测B地址{nvr_result_b}")
        ########################################################

        ### 实际地址 ######################################
        with open(f"./output/actrual_A_addr.txt", "a") as file:
            file.write(f"{actrual_addr.tolist()}\n")  
        print(f"实际A地址{actrual_addr.tolist()}")
        ########################################################
        np.set_printoptions(threshold=1000)

        
    mvin_instructions = access_matrix_np_b

    # for i in range(3):
    for i in range(len(access_matrix_np_b)-1):
        sp_predict_addr  =  sp.prefetch(mvin_instructions[i]) 
        actrual_addr     = mvin_instructions[i+1]
        
        np.set_printoptions(threshold=np.inf)
        ### 步长预取器结果 ######################################
        sp_result = []
        for _, _, predicted_address, _ in sp_predict_addr:
            sp_result.append(int(predicted_address))
        with open(f"./output/sp_B_predict_addr.txt", "a") as file:
            file.write(f"{sp_result}\n")  
        print(f"sp预测B地址{sp_result}" )
        ########################################################
        
        ### 实际地址 ######################################
        with open(f"./output/actrual_B_addr.txt", "a") as file:
            file.write(f"{actrual_addr.tolist()}\n")  
        print(f"实际B地址{actrual_addr.tolist()}")
        ########################################################
        np.set_printoptions(threshold=1000)



sp预测A地址[2, 4, 6, 30, 32, 42, 56, 74, 76, 78, 96, 110, 132, 144, 164, 166, 182, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
imp预测A地址[4, 5, 6, 0, 1, 0, 29, 0, 1, 2, 54, 57, 68, 73, 84, 85, 92]
imp预测B地址[688, 947, 1134, -1, 226, -1, 772, -1, 226, 287, 1125, 746, 1164, 1180, 776, 1090, 760]
dvr预测A地址[4, 5, 0, 0, 1, 0, 1, 29, 30, 0, 0, 1, 54, 55, 57, 58, 61, 68, 69, 73, 74, 79, 84, 85, 92, 93, 0]
dvr预测B地址[688, 947, -1, -1, 226, -1, 226, 772, 1455, -1, -1, 226, 1125, 442, 746, 942, 1072, 1164, 1558, 1180, 1207, 827, 776, 1090, 760, 923, -1]
nvr预测A地址[4, 5, 6, 29, 54, 57, 58, 61, 68, 73, 74, 79, 84, 85, 92, 93]
nvr预测B地址[688, 947, 1134, 772, 1125, 746, 942, 1072, 1164, 1180, 1207, 827, 776, 1090, 760, 923]
实际A地址[4, 5, 6, 29, 54, 57, 58, 61, 68, 73, 74, 79, 84, 85, 92, 93, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 

添加内存模型测试

In [6]:
import sys
sys.path.append("../")
from memory_model.memsim import MemSim

import argparse
import os
import configparser as cp


# parser = argparse.ArgumentParser()

# parser.add_argument('-c', metavar='Config file', type=str,
#                         default="./config/config.cfg",
#                         help="Path to the config file"
#                         )

# args = parser.parse_args()
# config = args.c
config = "./config/config.cfg"
if not os.path.exists(config):
    print("ERROR: Config file not found") 
    print("Input file:" + config)
    print('Exiting')
    exit()
else: 
    config_file = config

config = cp.ConfigParser()
config.read(config_file)

section = 'scratchpad_architecture_presets'

bank_num = int(config.get(section, 'Bank Num'))
bank_row = int(config.get(section, 'Bank Row per bank'))
dim = int(config.get(section, 'Dim'))
data_size = int(config.get(section, 'Data Size'))

section = 'l1cache_architecture_presets'
l1_size = int(config.get(section, 'Total Size'))
l1_cacheline_size = int(config.get(section, 'Cacheline Size'))
l1_way = int(config.get(section, 'Way of Associativity'))
l1_replacement = (config.get(section, 'Way of Replacement'))
l1_data_size   = int(config.get(section, 'Data Size'))

section = 'l2cache_architecture_presets'
l2_size = int(config.get(section, 'Total Size'))
l2_cacheline_size = int(config.get(section, 'Cacheline Size'))
l2_way = int(config.get(section, 'Way of Associativity'))
l2_replacement = (config.get(section, 'Way of Replacement'))
l2_data_size   = int(config.get(section, 'Data Size'))


sim = MemSim()
sim.set_params(bank_num, bank_row, dim, data_size, 
               l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
               l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
# for i in range(0, 0x10000, 0x4):
#     sim.spm_read(i)
# sim.print_info()
    
# for i in range(0, 0x10000, 0x4):
#     sim.cache_read(i)
# sim.print_info()

In [8]:
import ast
import numpy as np


def mem_test(test_case, format_byte):
    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
    
    
    A_predict_addr = []
    with open(f"./output/{test_case}_A_predict_addr.txt", 'r') as file:
        for line in file:
            # 去除换行符并转换为列表
            data_list = ast.literal_eval(line.strip())
                    # 检查长度并填充-1直到长度为64
            while len(data_list) < 64:
                data_list.append(-1)
            # 如果多于64个元素，则截取前64个元素
            # data_list = data_list[:64]
            A_predict_addr.append(data_list)
    A_predict_addr = np.array(A_predict_addr)
    

    B_predict_addr = []
    with open(f"./output/{test_case}_B_predict_addr.txt", 'r') as file:
        for line in file:
            data_list = ast.literal_eval(line.strip())
            while len(data_list) < 64:
                data_list.append(-1)
            B_predict_addr.append(data_list)
    B_predict_addr = np.array(B_predict_addr)

    A_actrual_addr = []
    with open(f"./output/actrual_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)

    
    vector_size = 128

    l1_miss_times = 0
    l1_miss_times_after_prefetch = 0
    l1_miss_iters = 0
    l1_miss_iters_after_prefetch = 0
    
    l2_miss_times = 0
    l2_miss_times_after_prefetch = 0
    l2_miss_iters = 0
    l2_miss_iters_after_prefetch = 0
    for i in range(A_predict_addr.shape[0]):
        for j in range(A_predict_addr.shape[1]): # 预取一轮
            if (A_predict_addr[i][j] != -1):
                sim.cache_read((A_predict_addr[i][j])*format_byte)
        l1miss, _ = sim.l1cache.print_info()
        l2miss, _ = sim.l2cache.print_info()
        if l1miss > l1_miss_times:
            l1_miss_iters += 1
            l1_miss_times = l1miss
        if l2miss > l2_miss_times:
            l2_miss_iters += 1
            l2_miss_times = l2miss
        # print(f"prefetch miss{miss_times}")
        
        for m in range(B_actrual_addr.shape[1]): # 实际访问一轮
            if (B_actrual_addr[i][m] != -1):
                sim.cache_read((A_predict_addr[i][m])*format_byte)
                # vector_mode    
                # for offset in range(vector_size):
                #     sim.spm_read((B_predict_addr[i][m]*vector_size+offset)*format_byte)
                        
        l1miss, _ = sim.l1cache.print_info()
        l2miss, _ = sim.l2cache.print_info()
        if l1miss > l1_miss_times:
            l1_miss_iters_after_prefetch += 1
            l1_miss_times_after_prefetch += l1miss - l1_miss_times
        if l2miss > l2_miss_times:
            l2_miss_iters_after_prefetch += 1
            l2_miss_times_after_prefetch += l2miss - l2_miss_times
        # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    print(f"l1_miss_times_after_prefetch={l1_miss_times_after_prefetch}, l1_miss_iters_after_prefetch={l1_miss_iters_after_prefetch}")
    print(f"l2_miss_times_after_prefetch={l2_miss_times_after_prefetch}, l2_miss_iters_after_prefetch={l2_miss_iters_after_prefetch}")
    sim.print_info()

print("=====sp test score========")
print("int8  ====================")
mem_test(test_case = "sp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "sp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "sp", format_byte=4)

print("=====imp test score========")
print("int8  ====================")
mem_test(test_case = "imp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "imp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "imp", format_byte=4)

print("=====dvr test score========")
print("int8  ====================")
mem_test(test_case = "dvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "dvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "dvr", format_byte=4)

print("=====nvr test score========")
print("int8  ====================")
mem_test(test_case = "nvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "nvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "nvr", format_byte=4)


=====sp test score========
int8  ====================
l1_miss_times_total=587, l1_miss_iters_total=270
l2_miss_times_total=379, l2_miss_iters_total=207
l1_miss_times_after_prefetch=48, l1_miss_iters_after_prefetch=45
l2_miss_times_after_prefetch=3, l2_miss_iters_after_prefetch=3
L1Cache hit 26802  miss 587 miss rate 0.021431179262504565
L2Cache hit 208  miss 379 miss rate 0.6445578231292517
Total miss rate 0.013813634252532705
fp16  ====================
l1_miss_times_total=1160, l1_miss_iters_total=387
l2_miss_times_total=717, l2_miss_iters_total=298
l1_miss_times_after_prefetch=106, l1_miss_iters_after_prefetch=95
l2_miss_times_after_prefetch=5, l2_miss_iters_after_prefetch=4
L1Cache hit 26229  miss 1160 miss rate 0.04235122307411464
L2Cache hit 443  miss 717 miss rate 0.6175710594315246
Total miss rate 0.026154889702101804
int32 ====================
l1_miss_times_total=2181, l1_miss_iters_total=564
l2_miss_times_total=1342, l2_miss_iters_total=440
l1_miss_times_after_prefetch=284, l1

In [33]:
import ast
import numpy as np


def mem_test(test_case, format_byte):
    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
    
    
    A_predict_addr = []
    with open(f"./output/{test_case}_A_predict_addr.txt", 'r') as file:
        for line in file:
            # 去除换行符并转换为列表
            data_list = ast.literal_eval(line.strip())
                    # 检查长度并填充-1直到长度为64
            while len(data_list) < 64:
                data_list.append(-1)
            # 如果多于64个元素，则截取前64个元素
            # data_list = data_list[:64]
            A_predict_addr.append(data_list)
    A_predict_addr = np.array(A_predict_addr)
    

    B_predict_addr = []
    with open(f"./output/{test_case}_B_predict_addr.txt", 'r') as file:
        for line in file:
            data_list = ast.literal_eval(line.strip())
            while len(data_list) < 64:
                data_list.append(-1)
            B_predict_addr.append(data_list)
    B_predict_addr = np.array(B_predict_addr)

    A_actrual_addr = []
    with open(f"./output/actrual_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)

    
    vector_size = 128

    l1_miss_times = 0
    l1_miss_times_after_prefetch = 0
    l1_miss_iters = 0
    l1_miss_iters_after_prefetch = 0
    
    l2_miss_times = 0
    l2_miss_times_after_prefetch = 0
    l2_miss_iters = 0
    l2_miss_iters_after_prefetch = 0
    for i in range(B_predict_addr.shape[0]):
        for j in range(B_predict_addr.shape[1]): # 预取一轮
            if (B_predict_addr[i][j] != -1):
                # sim.cache_read((B_predict_addr[i][j]*vector_size)*format_byte)
                # vector_mode    
                for offset in range(vector_size):
                    sim.spm_read((B_predict_addr[i][j]*vector_size+offset)*format_byte)
                    if offset % 16 == 15:
                        l1miss, _ = sim.l1cache.print_info()
                        l2miss, _ = sim.l2cache.print_info()
                        if l1miss > l1_miss_times:
                            l1_miss_iters += 1
                            l1_miss_times = l1miss
                        if l2miss > l2_miss_times:
                            l2_miss_iters += 1
                            l2_miss_times = l2miss
        # print(f"prefetch miss{miss_times}")
                    if (B_actrual_addr[i][j] != -1):
                        for offset in range(offset-15, offset):
                            sim.spm_read((B_actrual_addr[i][j]*vector_size+offset)*format_byte)
        
        # for m in range(B_actrual_addr.shape[1]): # 实际访问一轮
        #     if (B_actrual_addr[i][m] != -1):
        #         # sim.cache_read((B_predict_addr[i][m]*vector_size)*format_byte)
        #         # vector_mode    
        #         for offset in range(vector_size):
        #             sim.spm_read((B_predict_addr[i][m]*vector_size+offset)*format_byte)
                        
                    l1miss, _ = sim.l1cache.print_info()
                    l2miss, _ = sim.l2cache.print_info()
                    if l1miss > l1_miss_times:
                        l1_miss_iters_after_prefetch += 1
                        l1_miss_times_after_prefetch += l1miss - l1_miss_times
                    if l2miss > l2_miss_times:
                        l2_miss_iters_after_prefetch += 1
                        l2_miss_times_after_prefetch += l2miss - l2_miss_times
        # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    print(f"l1_miss_times_after_prefetch={l1_miss_times_after_prefetch}, l1_miss_iters_after_prefetch={l1_miss_iters_after_prefetch}")
    print(f"l2_miss_times_after_prefetch={l2_miss_times_after_prefetch}, l2_miss_iters_after_prefetch={l2_miss_iters_after_prefetch}")
    sim.print_info()

print("=====sp test score========")
print("int8  ====================")
mem_test(test_case = "sp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "sp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "sp", format_byte=4)

print("=====imp test score========")
print("int8  ====================")
mem_test(test_case = "imp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "imp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "imp", format_byte=4)

print("=====dvr test score========")
print("int8  ====================")
mem_test(test_case = "dvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "dvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "dvr", format_byte=4)

print("=====nvr test score========")
print("int8  ====================")
mem_test(test_case = "nvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "nvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "nvr", format_byte=4)


=====sp test score========
int8  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=517398, l2_miss_iters_total=332674
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=7568778, l2_miss_iters_after_prefetch=4932810
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 413639530  miss 517398 miss rate 0.0012492800766349124
Total miss rate 0.0
fp16  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=1166660, l2_miss_iters_total=746760
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=17023830, l2_miss_iters_after_prefetch=11075427
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 412990268  miss 1166660 miss rate 0.0028169515425395674
Total miss rate 0.0
int32 ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=2675708, l2_miss_iters_total=1624352
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l

In [34]:
import ast
import numpy as np


def mem_test(test_case, format_byte):
    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
    
    
    A_predict_addr = []
    with open(f"./output/{test_case}_A_predict_addr.txt", 'r') as file:
        for line in file:
            # 去除换行符并转换为列表
            data_list = ast.literal_eval(line.strip())
                    # 检查长度并填充-1直到长度为64
            while len(data_list) < 64:
                data_list.append(-1)
            # 如果多于64个元素，则截取前64个元素
            # data_list = data_list[:64]
            A_predict_addr.append(data_list)
    A_predict_addr = np.array(A_predict_addr)
    

    B_predict_addr = []
    with open(f"./output/{test_case}_B_predict_addr.txt", 'r') as file:
        for line in file:
            data_list = ast.literal_eval(line.strip())
            while len(data_list) < 64:
                data_list.append(-1)
            B_predict_addr.append(data_list)
    B_predict_addr = np.array(B_predict_addr)

    A_actrual_addr = []
    with open(f"./output/actrual_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)

    
    vector_size = 128

    l1_miss_times = 0
    l1_miss_times_after_prefetch = 0
    l1_miss_iters = 0
    l1_miss_iters_after_prefetch = 0
    
    l2_miss_times = 0
    l2_miss_times_after_prefetch = 0
    l2_miss_iters = 0
    l2_miss_iters_after_prefetch = 0
    for i in range(B_predict_addr.shape[0]):
        for j in range(B_predict_addr.shape[1]): # 预取一轮
            if (B_predict_addr[i][j] != -1):
                # sim.cache_read((B_predict_addr[i][j]*vector_size)*format_byte)
                # vector_mode    
                for offset in range(vector_size):
                    sim.spm_read((B_predict_addr[i][j]*vector_size+offset)*format_byte)
                    if offset % 64 == 63:
                        l1miss, _ = sim.l1cache.print_info()
                        l2miss, _ = sim.l2cache.print_info()
                        if l1miss > l1_miss_times:
                            l1_miss_iters += 1
                            l1_miss_times = l1miss
                        if l2miss > l2_miss_times:
                            l2_miss_iters += 1
                            l2_miss_times = l2miss
        # print(f"prefetch miss{miss_times}")
                    if (B_actrual_addr[i][j] != -1):
                        for offset in range(offset-15, offset):
                            sim.spm_read((B_actrual_addr[i][j]*vector_size+offset)*format_byte)
                    
                    l1miss, _ = sim.l1cache.print_info()
                    l2miss, _ = sim.l2cache.print_info()
                    if l1miss > l1_miss_times:
                        l1_miss_iters_after_prefetch += 1
                        l1_miss_times_after_prefetch += l1miss - l1_miss_times
                    if l2miss > l2_miss_times:
                        l2_miss_iters_after_prefetch += 1
                        l2_miss_times_after_prefetch += l2miss - l2_miss_times
        
        # for m in range(B_actrual_addr.shape[1]): # 实际访问一轮
        #     if (B_actrual_addr[i][m] != -1):
        #         # sim.cache_read((B_predict_addr[i][m]*vector_size)*format_byte)
        #         # vector_mode    
        #         for offset in range(vector_size):
        #             sim.spm_read((B_predict_addr[i][m]*vector_size+offset)*format_byte)
                        
   
        # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    print(f"l1_miss_times_after_prefetch={l1_miss_times_after_prefetch}, l1_miss_iters_after_prefetch={l1_miss_iters_after_prefetch}")
    print(f"l2_miss_times_after_prefetch={l2_miss_times_after_prefetch}, l2_miss_iters_after_prefetch={l2_miss_iters_after_prefetch}")
    sim.print_info()

print("=====sp test score========")
print("int8  ====================")
mem_test(test_case = "sp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "sp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "sp", format_byte=4)

print("=====imp test score========")
print("int8  ====================")
mem_test(test_case = "imp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "imp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "imp", format_byte=4)

print("=====dvr test score========")
print("int8  ====================")
mem_test(test_case = "dvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "dvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "dvr", format_byte=4)

print("=====nvr test score========")
print("int8  ====================")
mem_test(test_case = "nvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "nvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "nvr", format_byte=4)

=====sp test score========
int8  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=517398, l2_miss_iters_total=332674
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=32403882, l2_miss_iters_after_prefetch=20901162
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 413639530  miss 517398 miss rate 0.0012492800766349124
Total miss rate 0.0
fp16  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=1166660, l2_miss_iters_total=395786
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=56388758, l2_miss_iters_after_prefetch=24449014
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 412990268  miss 1166660 miss rate 0.0028169515425395674
Total miss rate 0.0
int32 ====================


In [28]:
import ast
import numpy as np


def mem_test(test_case, format_byte):
    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
    
    
    A_predict_addr = []
    with open(f"./output/{test_case}_A_predict_addr.txt", 'r') as file:
        for line in file:
            # 去除换行符并转换为列表
            data_list = ast.literal_eval(line.strip())
                    # 检查长度并填充-1直到长度为64
            while len(data_list) < 64:
                data_list.append(-1)
            # 如果多于64个元素，则截取前64个元素
            # data_list = data_list[:64]
            A_predict_addr.append(data_list)
    A_predict_addr = np.array(A_predict_addr)
    

    B_predict_addr = []
    with open(f"./output/{test_case}_B_predict_addr.txt", 'r') as file:
        for line in file:
            data_list = ast.literal_eval(line.strip())
            while len(data_list) < 64:
                data_list.append(-1)
            B_predict_addr.append(data_list)
    B_predict_addr = np.array(B_predict_addr)

    A_actrual_addr = []
    with open(f"./output/actrual_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)

    
    vector_size = 128

    l1_miss_times = 0
    l1_miss_times_after_prefetch = 0
    l1_miss_iters = 0
    l1_miss_iters_after_prefetch = 0
    
    l2_miss_times = 0
    l2_miss_times_after_prefetch = 0
    l2_miss_iters = 0
    l2_miss_iters_after_prefetch = 0
    for i in range(B_predict_addr.shape[0]):
        for j in range(B_predict_addr.shape[1]): # 预取一轮
            if (B_predict_addr[i][j] != -1):
                # sim.cache_read((B_predict_addr[i][j]*vector_size)*format_byte)
                # vector_mode    
                for offset in range(vector_size):
                    sim.spm_read((B_predict_addr[i][j]*vector_size+offset)*format_byte)
                l1miss, _ = sim.l1cache.print_info()
                l2miss, _ = sim.l2cache.print_info()
                if l1miss > l1_miss_times:
                    l1_miss_iters += 1
                    l1_miss_times = l1miss
                if l2miss > l2_miss_times:
                    l2_miss_iters += 1
                    l2_miss_times = l2miss
        # print(f"prefetch miss{miss_times}")
                if (B_actrual_addr[i][j] != -1):
                    for offset in range(offset-15, offset):
                        sim.spm_read((B_actrual_addr[i][j]*vector_size+offset)*format_byte)
                    
                l1miss, _ = sim.l1cache.print_info()
                l2miss, _ = sim.l2cache.print_info()
                if l1miss > l1_miss_times:
                    l1_miss_iters_after_prefetch += 1
                    l1_miss_times_after_prefetch += l1miss - l1_miss_times
                if l2miss > l2_miss_times:
                    l2_miss_iters_after_prefetch += 1
                    l2_miss_times_after_prefetch += l2miss - l2_miss_times
        
        # for m in range(B_actrual_addr.shape[1]): # 实际访问一轮
        #     if (B_actrual_addr[i][m] != -1):
        #         # sim.cache_read((B_predict_addr[i][m]*vector_size)*format_byte)
        #         # vector_mode    
        #         for offset in range(vector_size):
        #             sim.spm_read((B_predict_addr[i][m]*vector_size+offset)*format_byte)
                        
   
        # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    print(f"l1_miss_times_after_prefetch={l1_miss_times_after_prefetch}, l1_miss_iters_after_prefetch={l1_miss_iters_after_prefetch}")
    print(f"l2_miss_times_after_prefetch={l2_miss_times_after_prefetch}, l2_miss_iters_after_prefetch={l2_miss_iters_after_prefetch}")
    sim.print_info()

print("=====sp test score========")
print("int8  ====================")
mem_test(test_case = "sp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "sp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "sp", format_byte=4)

print("=====imp test score========")
print("int8  ====================")
mem_test(test_case = "imp", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "imp", format_byte=2)
print("int32 ====================")
mem_test(test_case = "imp", format_byte=4)

print("=====dvr test score========")
print("int8  ====================")
mem_test(test_case = "dvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "dvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "dvr", format_byte=4)

print("=====nvr test score========")
print("int8  ====================")
mem_test(test_case = "nvr", format_byte=1)
print("fp16  ====================")
mem_test(test_case = "nvr", format_byte=2)
print("int32 ====================")
mem_test(test_case = "nvr", format_byte=4)

=====sp test score========
int8  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=384740, l2_miss_iters_total=185398
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=103423, l2_miss_iters_after_prefetch=103423
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 30415702  miss 384741 miss rate 0.012491410838103503
Total miss rate 0.0
fp16  ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=733301, l2_miss_iters_total=193061
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=112113, l2_miss_iters_after_prefetch=112113
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 30067141  miss 733302 miss rate 0.02380816328491888
Total miss rate 0.0
int32 ====================
l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=1474473, l2_miss_iters_total=202152
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_a

KeyboardInterrupt: 

In [31]:
import multiprocessing
import itertools

import ast
import numpy as np


def mem_test(test_case, format_byte):
    print(f"====={test_case} test score ========")
    print(f"=====format_byte = {format_byte} ========")

    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)
    
    
    A_predict_addr = []
    with open(f"./output/{test_case}_A_predict_addr.txt", 'r') as file:
        for line in file:
            # 去除换行符并转换为列表
            data_list = ast.literal_eval(line.strip())
                    # 检查长度并填充-1直到长度为64
            while len(data_list) < 64:
                data_list.append(-1)
            # 如果多于64个元素，则截取前64个元素
            # data_list = data_list[:64]
            A_predict_addr.append(data_list)
    A_predict_addr = np.array(A_predict_addr)
    

    B_predict_addr = []
    with open(f"./output/{test_case}_B_predict_addr.txt", 'r') as file:
        for line in file:
            data_list = ast.literal_eval(line.strip())
            while len(data_list) < 64:
                data_list.append(-1)
            B_predict_addr.append(data_list)
    B_predict_addr = np.array(B_predict_addr)

    A_actrual_addr = []
    with open(f"./output/actrual_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)

    
    vector_size = 128

    l1_miss_times = 0
    l1_miss_times_after_prefetch = 0
    l1_miss_iters = 0
    l1_miss_iters_after_prefetch = 0
    
    l2_miss_times = 0
    l2_miss_times_after_prefetch = 0
    l2_miss_iters = 0
    l2_miss_iters_after_prefetch = 0
    for i in range(B_predict_addr.shape[0]):
        for j in range(B_predict_addr.shape[1]): # 预取一轮
            if (B_predict_addr[i][j] != -1):
                # sim.cache_read((B_predict_addr[i][j]*vector_size)*format_byte)
                # vector_mode    
                for offset in range(vector_size):
                    sim.spm_read((B_predict_addr[i][j]*vector_size+offset)*format_byte)
                    if offset % 32 == 31:
                        l1miss, _ = sim.l1cache.print_info()
                        l2miss, _ = sim.l2cache.print_info()
                        if l1miss > l1_miss_times:
                            l1_miss_iters += 1
                            l1_miss_times = l1miss
                        if l2miss > l2_miss_times:
                            l2_miss_iters += 1
                            l2_miss_times = l2miss
        # print(f"prefetch miss{miss_times}")
                    if (B_actrual_addr[i][j] != -1):
                        for offset in range(offset-31, offset):
                            sim.spm_read((B_actrual_addr[i][j]*vector_size+offset)*format_byte)
                    
                    l1miss, _ = sim.l1cache.print_info()
                    l2miss, _ = sim.l2cache.print_info()
                    if l1miss > l1_miss_times:
                        l1_miss_iters_after_prefetch += 1
                        l1_miss_times_after_prefetch += l1miss - l1_miss_times
                    if l2miss > l2_miss_times:
                        l2_miss_iters_after_prefetch += 1
                        l2_miss_times_after_prefetch += l2miss - l2_miss_times
        
        # for m in range(B_actrual_addr.shape[1]): # 实际访问一轮
        #     if (B_actrual_addr[i][m] != -1):
        #         # sim.cache_read((B_predict_addr[i][m]*vector_size)*format_byte)
        #         # vector_mode    
        #         for offset in range(vector_size):
        #             sim.spm_read((B_predict_addr[i][m]*vector_size+offset)*format_byte)
                        
   
        # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    print(f"l1_miss_times_after_prefetch={l1_miss_times_after_prefetch}, l1_miss_iters_after_prefetch={l1_miss_iters_after_prefetch}")
    print(f"l2_miss_times_after_prefetch={l2_miss_times_after_prefetch}, l2_miss_iters_after_prefetch={l2_miss_iters_after_prefetch}")
    sim.print_info()



def parallel_mem_test(args):
    test_case, format_byte = args

    mem_test(test_case, format_byte)


# 定义要处理的 test_case 列表和 format_byte
test_cases = ["sp", "imp", "dvr", "nvr"] 
format_byte = [1, 2, 4]
# 使用 itertools.product 生成所有可能的组合
args_list = list(itertools.product(test_cases, format_byte))
# 使用 multiprocessing.Pool 并行执行任务
with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
    pool.map(parallel_mem_test, args_list)

=====sp test score =============sp test score =============sp test score =============imp test score =============imp test score =============imp test score =============dvr test score =============dvr test score =============dvr test score ========
=====nvr test score =============nvr test score =============nvr test score ========






=====format_byte = 4 ========


=====format_byte = 1 ========
=====format_byte = 4 =============format_byte = 2 =============format_byte = 2 =============format_byte = 1 =============format_byte = 1 =============format_byte = 2 =============format_byte = 4 ========
=====format_byte = 2 =============format_byte = 1 ========
=====format_byte = 4 ========









l1_miss_times_total=0, l1_miss_iters_total=0
l2_miss_times_total=2533096, l2_miss_iters_total=697931
l1_miss_times_after_prefetch=0, l1_miss_iters_after_prefetch=0
l2_miss_times_after_prefetch=58598982, l2_miss_iters_after_prefetch=21360026
L1Cache hit 0  miss 0 miss rate 0.0
L2Cache hit 721385

No Prefetch Test

In [ ]:
import ast
import numpy as np


def mem_test_noprefetch(test_case, format_byte):
    sim = MemSim()
    sim.set_params(bank_num, bank_row, dim, data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, 32, l1_data_size, 
                l1_way, l1_size, l1_cacheline_size, l1_replacement, l1_data_size)

    
    A_actrual_addr = []
    with open(f"./output/actrual_{test_case}_A_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            A_actrual_addr.append(data_list)
    A_actrual_addr = np.array(A_actrual_addr)

    B_actrual_addr = []
    with open(f"./output/actrual_{test_case}_B_addr.txt", 'r') as file:
        for line in file:
            data_list = eval(line.strip())
            B_actrual_addr.append(data_list)
    B_actrual_addr = np.array(B_actrual_addr)



    l1_miss_times = 0
    l1_miss_iters = 0
    
    l2_miss_times = 0
    l2_miss_iters = 0

        
    for m in range(A_actrual_addr.shape[1]): # 实际访问一轮
        if (A_actrual_addr[i][m] != -1):
            sim.spm_read(A_actrual_addr[i][m]*format_byte)
    
    l1miss, _ = sim.l1cache.print_info()
    l2miss, _ = sim.l2cache.print_info()
    if l1miss > l1_miss_times:
        l1_miss_iters += 1
        l1_miss_times = l1miss
    if l2miss > l2_miss_times:
        l2_miss_iters += 1
        l2_miss_times = l2miss
    # print(f"after prefetch miss{miss_times_after_prefetch}")
            
    print(f"l1_miss_times_total={l1_miss_times}, l1_miss_iters_total={l1_miss_iters}")
    print(f"l2_miss_times_total={l2_miss_times}, l2_miss_iters_total={l2_miss_iters}")
    sim.print_info()



print("=====Dense test score========")
print("int8  ====================")
mem_test_noprefetch(test_case = "Dense", format_byte=1)
print("fp16  ====================")
mem_test_noprefetch(test_case = "Dense", format_byte=2)
print("int32  ====================")
mem_test_noprefetch(test_case = "Dense", format_byte=4)


print("=====sparse test score========")
print("int8  ====================")
mem_test_noprefetch(test_case = "sparse", format_byte=1)
print("fp16  ====================")
mem_test_noprefetch(test_case = "sparse", format_byte=2)
print("int32  ====================")
mem_test_noprefetch(test_case = "sparse", format_byte=4)
